# Lösningsförslag till Övningsuppgift - Stopped Flow
Författare: Björn Dahlgren, 2016-04-04

Detta lösningsförslag är skrivet i Python, se laborationshandledning för liknande exempel skrivna för Matlab.

In [ ]:
from __future__ import division, print_function
import numpy as np
from numpy.polynomial import polynomial as P
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import statsmodels.api as sm
%matplotlib inline

In [ ]:
n_conc, n_rep, n_t = 5, 10, 42
conc_B = np.linspace(1/10, n_conc/10, n_conc)  # molar
data = np.empty((n_conc, n_rep, n_t, 2))
for i in range(n_conc):
    for j in range(n_rep):
        data[i, j, :, :] = np.loadtxt('data/0.%d/%d.dat' % (i+1, j+1))

In [ ]:
def model(t, y0, k):
    return (y0**-2 + 6*k*t)**-0.5

In [ ]:
popt = np.empty((n_conc, n_rep, 2))  # y0, k
pcov = np.empty((n_conc, n_rep, 2, 2))
all_k = np.empty((n_conc, n_rep))
all_w = np.empty((n_conc, n_rep))
plt.figure(figsize=(16, 4))
for i in range(n_conc):
    ax = plt.subplot(1, n_conc, i+1)
    ax.set_yscale('log')
    for j in range(n_rep):
        t = data[i, j, :, 0]
        y = data[i, j, :, 1]
        popt[i, j, :], pcov[i, j, :, :] = curve_fit(model, t, y, [y[0], 1])
        m = model(t, *popt[i, j, :])
        plt.plot(t, y, 'k', alpha=0.4)
        plt.plot(t, m, 'b')
        all_k[i, j] = popt[i, j, 1]
        #all_w[i, j] = 1/pcov[i, j, 1, 1]             # varians i k
        #all_w[i, j] = 1/pcov[i, j, 0, 0]             # varians i y0
        all_w[i, j] = 1/np.average(np.square(y-m))    # MSD
        #all_w[i, j] = 1/(np.average(np.abs(y-m)))    # MAD

In [ ]:
k = np.empty(n_conc)
for i in range(n_conc):
    accum_k, accum_w = 0, 0
    for j in range(n_rep):
        w = all_w[i, j]
        accum_k += all_k[i, j]*w
        accum_w += w
    k[i] = accum_k/accum_w

In [ ]:
dk = np.empty_like(k)
for i in range(n_conc):
    accum_D, accum_w = 0, 0
    for j in range(n_rep):
        w = all_w[i, j]
        accum_D += w*(all_k[i, j] - k[i])**2
        accum_w += w
    dk[i] = accum_D/((n_rep - 1)*accum_w)

In [ ]:
plt.errorbar(conc_B, k, 3*dk**0.5, c='k', ls='None', marker='.')
p, cov = P.polyfit(conc_B, k, 1, w=1/dk, full=True)
plt.plot(conc_B, P.polyval(conc_B, p), 'b', label='$k_f$ = %.2f' % p[1])
plt.gca().set_xlim((0, conc_B[-1]*1.05))
plt.gca().set_ylim((0, k[-1]*1.1))
plt.legend(loc='best')
print(p)

In [ ]:
sm.WLS(k, sm.add_constant(conc_B), weights=1/dk).fit().summary()